<a href="https://colab.research.google.com/github/yakaralar/IE492/blob/master/promotion_day_exclusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import os
import glob
import shutil
os.listdir()

['.config', 'drive', 'sample_data']

In [3]:
os.listdir("drive/My Drive/colab_datas/IE 492")

['Comparison of methods.pdf',
 '1-s2.0-S1364032114000914-main.pdf',
 '1-s2.0-S0360544209002539-main.pdf',
 '1-s2.0-S1876610211043116-main.pdf',
 '1-s2.0-S0305048311001423-main.pdf',
 'weather_turkey_processed.zip',
 'Meta_L_makaleler',
 'all_data.feather',
 'UECM-15012015-20022020.csv',
 'RealTimeConsumption-01122014-09032020.csv',
 'updated_support-date_based_information.r',
 'ozel_gunler',
 'RealTimeConsumption_unholidayized.csv']

In [4]:
shutil.copy("drive/My Drive/colab_datas/IE 492/RealTimeConsumption-01122014-09032020.csv","./")

'./RealTimeConsumption-01122014-09032020.csv'

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas_profiling
%matplotlib inline
from tqdm import tqdm_notebook as tqdm

## Read cons df

In [44]:
# bruuuuuuuh
consumption_df=pd.read_csv("RealTimeConsumption-01122014-09032020.csv",encoding='latin-1')
consumption_df.columns=["Date","Hour","Consumption"]
consumption_df["Datetime"]=consumption_df["Date"]+" "+consumption_df["Hour"]
consumption_df["Consumption"]=consumption_df["Consumption"].apply(lambda x:x.replace(",","")).astype(float)
consumption_df["Datetime"]=pd.to_datetime(consumption_df["Datetime"],format="%d.%m.%Y %H:%M")
consumption_df

,Date,Hour,Consumption,Datetime
0,31.12.2015,00:00,29590.88,2015-12-31 00:00:00
1,31.12.2015,01:00,27785.43,2015-12-31 01:00:00
2,31.12.2015,02:00,26516.53,2015-12-31 02:00:00
3,31.12.2015,03:00,26091.62,2015-12-31 03:00:00
4,31.12.2015,04:00,25871.54,2015-12-31 04:00:00
...,...,...,...,...
36715,09.03.2020,19:00,0.00,2020-03-09 19:00:00
36716,09.03.2020,20:00,0.00,2020-03-09 20:00:00
36717,09.03.2020,21:00,0.00,2020-03-09 21:00:00
36718,09.03.2020,22:00,0.00,2020-03-09 22:00:00


In [45]:
consumption_df=consumption_df.loc[consumption_df["Datetime"]<pd.to_datetime("2020-03-04 00:00:00")]
consumption_df

,Date,Hour,Consumption,Datetime
0,31.12.2015,00:00,29590.88,2015-12-31 00:00:00
1,31.12.2015,01:00,27785.43,2015-12-31 01:00:00
2,31.12.2015,02:00,26516.53,2015-12-31 02:00:00
3,31.12.2015,03:00,26091.62,2015-12-31 03:00:00
4,31.12.2015,04:00,25871.54,2015-12-31 04:00:00
...,...,...,...,...
36571,03.03.2020,19:00,37936.79,2020-03-03 19:00:00
36572,03.03.2020,20:00,37553.37,2020-03-03 20:00:00
36573,03.03.2020,21:00,36581.73,2020-03-03 21:00:00
36574,03.03.2020,22:00,35434.36,2020-03-03 22:00:00


## Datelar promosyon tipleri koyulacak, sonradan lineer regresyon direkt vermeme gerek yok 

In [46]:
special_days_path="drive/My Drive/colab_datas/IE 492/ozel_gunler/*.csv"
special_day_df=pd.DataFrame()
exception=[]
for i in tqdm(glob.glob(special_days_path)):  
    a=pd.read_csv(i)
    try:
        a["special_day"]=np.full(len(a),a.columns[0])
        a.columns=["date","special_day"]
        special_day_df=pd.concat([special_day_df, a])
    except ValueError:
        exception.append(i)
        pass
special_day_df

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


,date,special_day
1,2010-06-20,babalargunu
2,2011-06-19,babalargunu
3,2012-06-17,babalargunu
4,2013-06-16,babalargunu
5,2014-06-15,babalargunu
...,...,...
6,2015-08-30,zaferbayrami
7,2016-08-30,zaferbayrami
8,2017-08-30,zaferbayrami
9,2018-08-30,zaferbayrami


In [0]:
special_day_df["is"]=np.ones(len(special_day_df))
special_day_df=pd.pivot_table(special_day_df,index="date",columns="special_day").fillna(0).astype(int)
special_day_df.columns=['_'.join(i) for i in special_day_df.columns]
special_day_df=special_day_df.reset_index()



## LR Date categorical and spec days


In [112]:
cons_lagged=consumption_df.copy()
cons_lagged["Consumption_2_lag"]=cons_lagged["Consumption"].shift(24*2)
cons_lagged["Consumption_7_lag"]=cons_lagged["Consumption"].shift(24*7)
cons_lagged["Hour"]=cons_lagged["Hour"].apply(lambda x:x.split(":")[0]).astype(int)
cons_lagged["Day"]=cons_lagged["Date"].apply(lambda x:x.split(".")[0]).astype(int)
cons_lagged["Month"]=cons_lagged["Date"].apply(lambda x:x.split(".")[1]).astype(int)
cons_lagged["weekday"]=pd.to_datetime(cons_lagged["Date"]).apply(lambda x: x.weekday()).astype(int)
cons_lagged=cons_lagged.dropna()
cons_lagged

,Date,Hour,Consumption,Datetime,Consumption_2_lag,Consumption_7_lag,Day,Month,weekday
168,07.01.2016,0,28763.95,2016-01-07 00:00:00,29244.38,29590.88,7,1,4
169,07.01.2016,1,27284.84,2016-01-07 01:00:00,27637.23,27785.43,7,1,4
170,07.01.2016,2,26321.95,2016-01-07 02:00:00,26335.78,26516.53,7,1,4
171,07.01.2016,3,25748.49,2016-01-07 03:00:00,25870.23,26091.62,7,1,4
172,07.01.2016,4,25636.58,2016-01-07 04:00:00,25975.07,25871.54,7,1,4
...,...,...,...,...,...,...,...,...,...
36571,03.03.2020,19,37936.79,2020-03-03 19:00:00,35049.21,39283.70,3,3,1
36572,03.03.2020,20,37553.37,2020-03-03 20:00:00,34901.84,38791.40,3,3,1
36573,03.03.2020,21,36581.73,2020-03-03 21:00:00,34225.14,37588.86,3,3,1
36574,03.03.2020,22,35434.36,2020-03-03 22:00:00,33257.06,36290.64,3,3,1


In [0]:
# new_dates=[]
# for date_ in special_day_df["date"].values:
#     new_dates=new_dates+[date_ + " 0{}:00".format(str(i)) if i<10 else date_ + " {}:00".format(str(i)) for i in range(24)]
# new_dates=pd.DataFrame(new_dates,columns=["datetime"])
# new_dates["date"]=new_dates["datetime"].apply(lambda x:x.split(" ")[0])
# new_dates

In [0]:
# special_day_df=pd.merge(new_dates,special_day_df,on="date",how="left")

In [0]:
# special_day_df["datetime"]=pd.to_datetime(special_day_df["datetime"],format="%Y-%m-%d %H:%M")
# special_day_df["datetime"].dtype

In [116]:
special_day_df["date"]=special_day_df["date"].apply(lambda x:'.'.join(list(reversed(x.split("-")))))
special_day_df

,date,is_annelergunu,is_arife,is_babalargunu,is_cocukbayrami,is_cumhuriyet,is_elections,is_examination,is_genclikbayrami,is_holiday,is_iscibayrami,is_kadinlargunu,is_kandil,is_kurbanbayrami,is_kurbanbayramilkgun,is_kurbanoncesigunler,is_onbestemmuz,is_outlierdays,is_ramazanbayrami,is_ramazanbayramilkgun,is_ramazanbayramisonrasi,is_ramazangunler,is_ramazanilkgun,is_ramazanoncesigunler,is_school_days,is_school_end,is_school_start,is_semester,is_semester_days,is_sevgililergunu,is_single_day_holidays,is_summerTimes,is_summer_break,is_uzunbayram,is_winterTimes,is_winter_break,is_winter_break_schools,is_yilbasi,is_yilbasioncesi,is_zaferbayrami
0,01.01.2010,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0
1,23.01.2010,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2,24.01.2010,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
3,25.01.2010,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
4,26.01.2010,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2889,28.02.2020,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0
2890,29.02.2020,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0
2891,01.03.2020,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2892,02.03.2020,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


In [117]:
cons_lagged=pd.merge(cons_lagged,special_day_df,left_on="Date",right_on="date",how="left").drop(columns=["date"]).fillna(0)
cons_lagged

,Date,Hour,Consumption,Datetime,Consumption_2_lag,Consumption_7_lag,Day,Month,weekday,is_annelergunu,is_arife,is_babalargunu,is_cocukbayrami,is_cumhuriyet,is_elections,is_examination,is_genclikbayrami,is_holiday,is_iscibayrami,is_kadinlargunu,is_kandil,is_kurbanbayrami,is_kurbanbayramilkgun,is_kurbanoncesigunler,is_onbestemmuz,is_outlierdays,is_ramazanbayrami,is_ramazanbayramilkgun,is_ramazanbayramisonrasi,is_ramazangunler,is_ramazanilkgun,is_ramazanoncesigunler,is_school_days,is_school_end,is_school_start,is_semester,is_semester_days,is_sevgililergunu,is_single_day_holidays,is_summerTimes,is_summer_break,is_uzunbayram,is_winterTimes,is_winter_break,is_winter_break_schools,is_yilbasi,is_yilbasioncesi,is_zaferbayrami
0,07.01.2016,0,28763.95,2016-01-07 00:00:00,29244.38,29590.88,7,1,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,07.01.2016,1,27284.84,2016-01-07 01:00:00,27637.23,27785.43,7,1,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,07.01.2016,2,26321.95,2016-01-07 02:00:00,26335.78,26516.53,7,1,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,07.01.2016,3,25748.49,2016-01-07 03:00:00,25870.23,26091.62,7,1,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,07.01.2016,4,25636.58,2016-01-07 04:00:00,25975.07,25871.54,7,1,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36403,03.03.2020,19,37936.79,2020-03-03 19:00:00,35049.21,39283.70,3,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
36404,03.03.2020,20,37553.37,2020-03-03 20:00:00,34901.84,38791.40,3,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
36405,03.03.2020,21,36581.73,2020-03-03 21:00:00,34225.14,37588.86,3,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
36406,03.03.2020,22,35434.36,2020-03-03 22:00:00,33257.06,36290.64,3,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


In [0]:
# from sklearn.preprocessing import OneHotEncoder
# from scipy.sparse import hstack

# cat_features=["Hour","Day","Month","weekday"]
# linear_enc=OneHotEncoder()
# one_hot_encoded=linear_enc.fit_transform(cons_lagged[cat_features]).toarray()
# normal_feats=cons_lagged.columns.difference(cat_features).tolist()
# cons_lagged=np.hstack([cons_lagged[normal_feats],one_hot_encoded])
# cons_lagged=pd.DataFrame(cons_lagged,columns=normal_feats+np.concatenate([[cat_features[i]+"_"+str(j) for j in cats_] for i,cats_ in enumerate(linear_enc.categories_)]).tolist())

In [119]:
cons_lagged[cons_lagged["is_onbestemmuz"]!=0]

,Date,Hour,Consumption,Datetime,Consumption_2_lag,Consumption_7_lag,Day,Month,weekday,is_annelergunu,is_arife,is_babalargunu,is_cocukbayrami,is_cumhuriyet,is_elections,is_examination,is_genclikbayrami,is_holiday,is_iscibayrami,is_kadinlargunu,is_kandil,is_kurbanbayrami,is_kurbanbayramilkgun,is_kurbanoncesigunler,is_onbestemmuz,is_outlierdays,is_ramazanbayrami,is_ramazanbayramilkgun,is_ramazanbayramisonrasi,is_ramazangunler,is_ramazanilkgun,is_ramazanoncesigunler,is_school_days,is_school_end,is_school_start,is_semester,is_semester_days,is_sevgililergunu,is_single_day_holidays,is_summerTimes,is_summer_break,is_uzunbayram,is_winterTimes,is_winter_break,is_winter_break_schools,is_yilbasi,is_yilbasioncesi,is_zaferbayrami
13296,15.07.2017,0,36615.48,2017-07-15 00:00:00,36908.27,35358.92,15,7,5,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0
13297,15.07.2017,1,34853.09,2017-07-15 01:00:00,35328.87,33701.07,15,7,5,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0
13298,15.07.2017,2,33479.85,2017-07-15 02:00:00,33858.35,32315.85,15,7,5,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0
13299,15.07.2017,3,32413.69,2017-07-15 03:00:00,33065.40,31474.32,15,7,5,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0
13300,15.07.2017,4,31695.45,2017-07-15 04:00:00,32442.87,30856.12,15,7,5,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30835,15.07.2019,19,34881.10,2019-07-15 19:00:00,36941.04,40928.17,15,7,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0
30836,15.07.2019,20,36159.92,2019-07-15 20:00:00,38055.13,41707.40,15,7,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0
30837,15.07.2019,21,36525.52,2019-07-15 21:00:00,38062.00,42080.98,15,7,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0
30838,15.07.2019,22,35391.93,2019-07-15 22:00:00,37203.59,40968.00,15,7,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0


In [0]:
cons_lagged_unmodif=cons_lagged.copy()
train_df=cons_lagged.loc[cons_lagged["Datetime"]<=pd.to_datetime("2019-03-03 23:00",format="%Y-%m-%d %H:%M")]
valid_df=cons_lagged.loc[cons_lagged["Datetime"]>pd.to_datetime("2019-03-03 23:00",format="%Y-%m-%d %H:%M")]

In [0]:
from sklearn.linear_model import LinearRegression

cons_lagged=cons_lagged.dropna()

cols=train_df.columns.difference(["Date","Consumption","Datetime"]).values

model=LinearRegression()
model.fit(train_df[cols].values,train_df["Consumption"].values.reshape(-1, 1))
preds=model.predict(valid_df[cols].values)

In [96]:
{train_df[cols].columns[i]:model.coef_[0][i] for i in range(len(model.coef_[0]))}

{'Consumption_2_lag': 0.21800039386395403,
 'Consumption_7_lag': 0.6876580495541188,
 'Day': 0.9591225903880888,
 'Hour': 34.08751764138581,
 'Month': 0.10131676817032934,
 'is_annelergunu': -897.2666870006713,
 'is_arife': 2854.710643474527,
 'is_babalargunu': 1039.7097437839518,
 'is_cocukbayrami': 663.9488426947134,
 'is_cumhuriyet': 194.00404628818865,
 'is_elections': -2906.554455676329,
 'is_examination': -262.4051526548538,
 'is_genclikbayrami': 333.76306164385704,
 'is_holiday': -4756.298130785115,
 'is_iscibayrami': 357.1222220991899,
 'is_kadinlargunu': -313.01092316602245,
 'is_kandil': -100.49971943546049,
 'is_kurbanbayrami': -2309.6714625414997,
 'is_kurbanbayramilkgun': -3270.791358279781,
 'is_kurbanoncesigunler': -1245.4043563385997,
 'is_onbestemmuz': 1013.15788870286,
 'is_outlierdays': 348.52419086616294,
 'is_ramazanbayrami': -4557.726958432488,
 'is_ramazanbayramilkgun': -1909.5083942894578,
 'is_ramazanbayramisonrasi': 3951.8151173725128,
 'is_ramazangunler': -17

In [0]:
sp_days_index=[i for i,col in enumerate(train_df[cols].columns) if col.startswith("is_")]

In [98]:
cons_lagged=pd.merge(consumption_df,special_day_df,left_on="Date",right_on="date",how="left").drop(columns="date").fillna(0)
cons_lagged

,Date,Hour,Consumption,Datetime,is_annelergunu,is_arife,is_babalargunu,is_cocukbayrami,is_cumhuriyet,is_elections,is_examination,is_genclikbayrami,is_holiday,is_iscibayrami,is_kadinlargunu,is_kandil,is_kurbanbayrami,is_kurbanbayramilkgun,is_kurbanoncesigunler,is_onbestemmuz,is_outlierdays,is_ramazanbayrami,is_ramazanbayramilkgun,is_ramazanbayramisonrasi,is_ramazangunler,is_ramazanilkgun,is_ramazanoncesigunler,is_school_days,is_school_end,is_school_start,is_semester,is_semester_days,is_sevgililergunu,is_single_day_holidays,is_summerTimes,is_summer_break,is_uzunbayram,is_winterTimes,is_winter_break,is_winter_break_schools,is_yilbasi,is_yilbasioncesi,is_zaferbayrami
0,31.12.2015,00:00,29590.88,2015-12-31 00:00:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,31.12.2015,01:00,27785.43,2015-12-31 01:00:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,31.12.2015,02:00,26516.53,2015-12-31 02:00:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,31.12.2015,03:00,26091.62,2015-12-31 03:00:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,31.12.2015,04:00,25871.54,2015-12-31 04:00:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36571,03.03.2020,19:00,37936.79,2020-03-03 19:00:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
36572,03.03.2020,20:00,37553.37,2020-03-03 20:00:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
36573,03.03.2020,21:00,36581.73,2020-03-03 21:00:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
36574,03.03.2020,22:00,35434.36,2020-03-03 22:00:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


In [99]:
special_days_effect_dict=dict(zip(train_df[cols].columns[sp_days_index].tolist(),model.coef_[0][sp_days_index].tolist()))
for key_ in special_days_effect_dict.keys():
    cons_lagged[key_]=cons_lagged[key_]*special_days_effect_dict[key_]
cons_lagged["Consumption"]=cons_lagged["Consumption"]-cons_lagged[list(special_days_effect_dict.keys())].sum(axis=1)
cons_lagged["Consumption"]

0        29745.882083
1        27940.432083
2        26671.532083
3        26246.622083
4        26026.542083
             ...     
36571    37743.533050
36572    37360.113050
36573    36388.473050
36574    35241.103050
36575    33633.743050
Name: Consumption, Length: 36576, dtype: float64

In [100]:
special_days_effect_dict

{'is_annelergunu': -897.2666870006713,
 'is_arife': 2854.710643474527,
 'is_babalargunu': 1039.7097437839518,
 'is_cocukbayrami': 663.9488426947134,
 'is_cumhuriyet': 194.00404628818865,
 'is_elections': -2906.554455676329,
 'is_examination': -262.4051526548538,
 'is_genclikbayrami': 333.76306164385704,
 'is_holiday': -4756.298130785115,
 'is_iscibayrami': 357.1222220991899,
 'is_kadinlargunu': -313.01092316602245,
 'is_kandil': -100.49971943546049,
 'is_kurbanbayrami': -2309.6714625414997,
 'is_kurbanbayramilkgun': -3270.791358279781,
 'is_kurbanoncesigunler': -1245.4043563385997,
 'is_onbestemmuz': 1013.15788870286,
 'is_outlierdays': 348.52419086616294,
 'is_ramazanbayrami': -4557.726958432488,
 'is_ramazanbayramilkgun': -1909.5083942894578,
 'is_ramazanbayramisonrasi': 3951.8151173725128,
 'is_ramazangunler': -171.35867354231843,
 'is_ramazanilkgun': 116.75644025975001,
 'is_ramazanoncesigunler': -199.08773425565232,
 'is_school_days': -193.25695039079022,
 'is_school_end': 214.557

In [102]:
sorted(cons_lagged[list(special_days_effect_dict.keys())].sum(axis=1).unique())

[-13622.90595153432,
 -11741.026339440707,
 -11713.397557244862,
 -10421.683353704902,
 -10131.72005230478,
 -9030.045587312652,
 -8512.174959415444,
 -8470.234981160926,
 -7472.465215631493,
 -6860.928694024999,
 -6264.1194635244565,
 -5828.822989483236,
 -5764.810266213697,
 -5480.298798617073,
 -4560.359842042932,
 -3099.811406067119,
 -2819.407880782979,
 -2703.7794635989712,
 -2680.4203031436386,
 -2676.050402885637,
 -2649.66974417291,
 -2644.450744698987,
 -2570.731141569577,
 -2481.3325688879863,
 -2399.372468027259,
 -2373.5936825481144,
 -2275.005667727923,
 -2174.505948292462,
 -1437.3358563444326,
 -1404.2653878343103,
 -1289.6113716471139,
 -1279.016938029618,
 -1261.88231093378,
 -830.1898220913672,
 -637.415507649536,
 -613.2923631465101,
 -506.2678735568127,
 -492.844404081903,
 -481.6656312252044,
 -465.11534336856914,
 -455.662103045644,
 -443.5542265364414,
 -392.34468464644254,
 -364.6156239331086,
 -293.7566698262507,
 -255.50180234498526,
 -247.85918367335864,
 -2

In [103]:
cons_lagged

,Date,Hour,Consumption,Datetime,is_annelergunu,is_arife,is_babalargunu,is_cocukbayrami,is_cumhuriyet,is_elections,is_examination,is_genclikbayrami,is_holiday,is_iscibayrami,is_kadinlargunu,is_kandil,is_kurbanbayrami,is_kurbanbayramilkgun,is_kurbanoncesigunler,is_onbestemmuz,is_outlierdays,is_ramazanbayrami,is_ramazanbayramilkgun,is_ramazanbayramisonrasi,is_ramazangunler,is_ramazanilkgun,is_ramazanoncesigunler,is_school_days,is_school_end,is_school_start,is_semester,is_semester_days,is_sevgililergunu,is_single_day_holidays,is_summerTimes,is_summer_break,is_uzunbayram,is_winterTimes,is_winter_break,is_winter_break_schools,is_yilbasi,is_yilbasioncesi,is_zaferbayrami
0,31.12.2015,00:00,29745.882083,2015-12-31 00:00:00,-0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,0.0,-0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,0.0,-0.0,-0.0,0.0,-0.0,0.0,-0.0,-193.25695,0.0,0.0,-0.0,0.00000,0.0,0.0,-0.0,0.0,-0.0,-0.0,-0.0,0.0,-0.0,38.254867,0.0
1,31.12.2015,01:00,27940.432083,2015-12-31 01:00:00,-0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,0.0,-0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,0.0,-0.0,-0.0,0.0,-0.0,0.0,-0.0,-193.25695,0.0,0.0,-0.0,0.00000,0.0,0.0,-0.0,0.0,-0.0,-0.0,-0.0,0.0,-0.0,38.254867,0.0
2,31.12.2015,02:00,26671.532083,2015-12-31 02:00:00,-0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,0.0,-0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,0.0,-0.0,-0.0,0.0,-0.0,0.0,-0.0,-193.25695,0.0,0.0,-0.0,0.00000,0.0,0.0,-0.0,0.0,-0.0,-0.0,-0.0,0.0,-0.0,38.254867,0.0
3,31.12.2015,03:00,26246.622083,2015-12-31 03:00:00,-0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,0.0,-0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,0.0,-0.0,-0.0,0.0,-0.0,0.0,-0.0,-193.25695,0.0,0.0,-0.0,0.00000,0.0,0.0,-0.0,0.0,-0.0,-0.0,-0.0,0.0,-0.0,38.254867,0.0
4,31.12.2015,04:00,26026.542083,2015-12-31 04:00:00,-0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,0.0,-0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,0.0,-0.0,-0.0,0.0,-0.0,0.0,-0.0,-193.25695,0.0,0.0,-0.0,0.00000,0.0,0.0,-0.0,0.0,-0.0,-0.0,-0.0,0.0,-0.0,38.254867,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36571,03.03.2020,19:00,37743.533050,2020-03-03 19:00:00,-0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,0.0,-0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,0.0,-0.0,-0.0,0.0,-0.0,0.0,-0.0,-0.00000,0.0,0.0,-0.0,193.25695,0.0,0.0,-0.0,0.0,-0.0,-0.0,-0.0,0.0,-0.0,0.000000,0.0
36572,03.03.2020,20:00,37360.113050,2020-03-03 20:00:00,-0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,0.0,-0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,0.0,-0.0,-0.0,0.0,-0.0,0.0,-0.0,-0.00000,0.0,0.0,-0.0,193.25695,0.0,0.0,-0.0,0.0,-0.0,-0.0,-0.0,0.0,-0.0,0.000000,0.0
36573,03.03.2020,21:00,36388.473050,2020-03-03 21:00:00,-0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,0.0,-0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,0.0,-0.0,-0.0,0.0,-0.0,0.0,-0.0,-0.00000,0.0,0.0,-0.0,193.25695,0.0,0.0,-0.0,0.0,-0.0,-0.0,-0.0,0.0,-0.0,0.000000,0.0
36574,03.03.2020,22:00,35241.103050,2020-03-03 22:00:00,-0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,0.0,-0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,0.0,-0.0,-0.0,0.0,-0.0,0.0,-0.0,-0.00000,0.0,0.0,-0.0,193.25695,0.0,0.0,-0.0,0.0,-0.0,-0.0,-0.0,0.0,-0.0,0.000000,0.0


In [0]:
cons_lagged.to_csv("drive/My Drive/colab_datas/IE 492/RealTimeConsumption_unholidayized.csv",index=False)

# Burada interpolasyon yapılacak gecen haftaki saatler özelinde 
#### U U U UWU UWAAA UWU https://www.youtube.com/watch?v=0OZyleriwRU

In [121]:
cons_lagged=cons_lagged_unmodif.copy()


,Date,Hour,Consumption,Datetime,Consumption_2_lag,Consumption_7_lag,Day,Month,weekday,is_annelergunu,is_arife,is_babalargunu,is_cocukbayrami,is_cumhuriyet,is_elections,is_examination,is_genclikbayrami,is_holiday,is_iscibayrami,is_kadinlargunu,is_kandil,is_kurbanbayrami,is_kurbanbayramilkgun,is_kurbanoncesigunler,is_onbestemmuz,is_outlierdays,is_ramazanbayrami,is_ramazanbayramilkgun,is_ramazanbayramisonrasi,is_ramazangunler,is_ramazanilkgun,is_ramazanoncesigunler,is_school_days,is_school_end,is_school_start,is_semester,is_semester_days,is_sevgililergunu,is_single_day_holidays,is_summerTimes,is_summer_break,is_uzunbayram,is_winterTimes,is_winter_break,is_winter_break_schools,is_yilbasi,is_yilbasioncesi,is_zaferbayrami
0,07.01.2016,0,28763.95,2016-01-07 00:00:00,29244.38,29590.88,7,1,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,07.01.2016,1,27284.84,2016-01-07 01:00:00,27637.23,27785.43,7,1,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,07.01.2016,2,26321.95,2016-01-07 02:00:00,26335.78,26516.53,7,1,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,07.01.2016,3,25748.49,2016-01-07 03:00:00,25870.23,26091.62,7,1,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,07.01.2016,4,25636.58,2016-01-07 04:00:00,25975.07,25871.54,7,1,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36403,03.03.2020,19,37936.79,2020-03-03 19:00:00,35049.21,39283.70,3,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
36404,03.03.2020,20,37553.37,2020-03-03 20:00:00,34901.84,38791.40,3,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
36405,03.03.2020,21,36581.73,2020-03-03 21:00:00,34225.14,37588.86,3,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
36406,03.03.2020,22,35434.36,2020-03-03 22:00:00,33257.06,36290.64,3,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
